In [1]:
DATA_NAME = 'syntheticMRI3D-full' 
TRANSFORM = 'wavelet'
CHANNEL = 'daa'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_daa_wavelet_full_syntheticMRI3D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) <100:
            X[:, i] = np.zeros(bootstrap_size)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8
2,0.0,0.00,0.00,0.00,0.00,0.00,0.00
3,0.0,6901357.58,-288.36,15.61,-0.87,-5.09,1.24
4,0.0,-288.36,4597811.53,-35.86,15.36,9.21,1.54
5,0.0,15.61,-35.86,1341900.23,2.57,0.07,1.06
6,0.0,-0.87,15.36,2.57,182291.18,-0.03,0.45
7,0.0,-5.09,9.21,0.07,-0.03,15689.51,-0.20
8,0.0,1.24,1.54,1.06,0.45,-0.20,584.61


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

/var/folders/0c/5hhs4r6n7pnc4jcl295d7j_c0000gn/T/ipykernel_22074/980427090.py:1: RuntimeWarning: invalid value encountered in divide
  corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))


,2,3,4,5,6,7,8
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.00000,-0.00005,0.00001,-0.00000,-0.00002,0.00002
4,NaN,-0.00005,1.00000,-0.00001,0.00002,0.00003,0.00003
5,NaN,0.00001,-0.00001,1.00000,0.00001,0.00000,0.00004
6,NaN,-0.00000,0.00002,0.00001,1.00000,-0.00000,0.00004
7,NaN,-0.00002,0.00003,0.00000,-0.00000,1.00000,-0.00007
8,NaN,0.00002,0.00003,0.00004,0.00004,-0.00007,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

412.4092231988191

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.95505144e+12 2.88792262e+12 2.39115629e+11 4.15177388e+09
 2.73379778e+07 2.84705405e+04 0.00000000e+00]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.0,0.981842,-0.187556,-0.028201,-0.003670,-0.000317,-0.000012
1,0.0,0.185183,0.980153,-0.070242,-0.008562,-0.000734,-0.000027
2,0.0,0.040607,0.063433,0.996528,-0.035351,-0.002973,-0.000110
3,-0.0,0.006610,0.009924,0.034448,0.998914,-0.029000,-0.001067
4,0.0,0.000759,0.001136,0.003899,0.028843,0.999400,-0.018699
5,-0.0,0.000042,0.000063,0.000217,0.001601,0.018660,0.999825
6,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.0,
 0.8148167687543906,
 0.9365673392225877,
 0.9655520802463621,
 0.9711570767189046,
 0.9813399518939445,
 1.0]